## Create a DatataFrame from a list of dicts

In [2]:
import pandas as pd

In [3]:
dfmm=pd.DataFrame( [ {'key':'A','data':1} , 
                     {'key':'B','data':2} ,
                     {'key':'C','data':3} ,                       
                     {'key':'A','data':4} ,
                     {'key':'B', 'data':5},
                     {'key':'C','data':6}
                   ]  
                 )


In [4]:
print(dfmm)

   data key
0     1   A
1     2   B
2     3   C
3     4   A
4     5   B
5     6   C


In [5]:
dfmm.groupby('key').sum()

,data
key,
A,5
B,7
C,9


In [6]:
dfmm.groupby('key').count()

,data
key,
A,2
B,2
C,2


## example : analyze the data 

In [1]:
import json
import requests
res=requests.get("https://od.cdc.gov.tw/eic/NHI_EnteroviralInfection.json")

In [2]:
result=json.loads(res.text)
print(type(result))

<class 'list'>


In [3]:
len(result)

112173

In [4]:
result[0]

{'健保就診總人次': '105',
 '就診類別': '住院',
 '年': '2008',
 '年齡別': '0-2',
 '縣市': '台中市',
 '腸病毒健保就診人次': '0',
 '週': '14'}

In [5]:
import pandas as pd
df=pd.DataFrame(result)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112173 entries, 0 to 112172
Data columns (total 7 columns):
健保就診總人次      112173 non-null object
就診類別         112173 non-null object
年            112173 non-null object
年齡別          112173 non-null object
縣市           112173 non-null object
腸病毒健保就診人次    112173 non-null int64
週            112173 non-null object
dtypes: int64(1), object(6)
memory usage: 6.0+ MB


# transform the string into integer

In [12]:
df['腸病毒健保就診人次'] = df['腸病毒健保就診人次'].astype(int)

In [13]:
df.dtypes

健保就診總人次      object
就診類別         object
年            object
年齡別          object
縣市           object
腸病毒健保就診人次     int64
週            object
dtype: object

## 　groupby('年') 分析：每年Taiwan（2008~2018) 腸病毒健保就診人次多少?

In [9]:
df.groupby('年')[['腸病毒健保就診人次']].sum()

,腸病毒健保就診人次
年,
2008,360000
2009,270178
2010,855397
2011,409542
2012,483404
2013,578879
2014,503170
2015,535773
2016,590212


## Let's check the above one-line code : 

In [14]:
leng=len(df['縣市'])
print(leng)

112173


In [15]:


d=dict()
for i in range(leng):
    key=df['年'][i]
    d[key]=0
    
for i in range(leng):
        key=df['年'][i]
        d[key]=d[key]+df['腸病毒健保就診人次'][i]
print(d)

{'2008': 360000, '2009': 270178, '2010': 855397, '2011': 409542, '2012': 483404, '2013': 578879, '2014': 503170, '2015': 535773, '2016': 590212, '2017': 455533, '2018': 90211}


## 樞紐分析
## 不同縣市每年腸病毒就診類別(住院,門診)的總人次是多少 ? 

In [16]:
df.pivot_table(values='腸病毒健保就診人次',index=['年','就診類別'],columns='縣市',aggfunc='sum')

縣市           南投縣     台中市    台北市    台南市   台東縣    嘉義市   嘉義縣    基隆市    宜蘭縣  \
年    就診類別                                                                 
2008 住院       14     765    739     62     6     11    30     78     13   
     門診    10056   55128  26905  41456  2934   8274  4016   4760   7288   
2009 住院      153     986    820    115    59    186   237    124    297   
     門診     6645   46014  17825  24913  2806   6779  2948   4229   7042   
2010 住院      352    2327   2305    251    68    408   588    388   1279   
     門診    23468  137646  68862  72263  4521  19335  8712  13457  14426   
2011 住院      165     891   1110    142    63    189   339    207    783   
     門診     8758   55265  34228  36128  2371  10101  5179   6580   6767   
2012 住院      128    1193   1256    212    68    149   264    206    583   
     門診     8768   68450  44295  39727  2339   8245  3790   8448   9785   
2013 住院      271    1759   1232    207    90    207   353    187    901   
     門診    15680   90260  45548  51344  3896  10392  5209   7438  10585   
2014 住院      169    1276    958    110    59    160   283    175    660   
     門診    12413   73386  45480  36193  2662  10491  6071   8024  11156   
2015 住院      146    1175    801    135    57    150   290    196    567   
     門診    10953   80743  47070  39807  2792   9721  6311   9063  10962   
2016 住院      277    1389    857    182    67     91   274    166    469   
     門診    13942   96622  49049  51394  3403   8643  5977   8782  11312   
2017 住院      194     991    771     80    37     79   252    148    404   
     門診     9115   65744  44981  28147  3057   6296  4265   7399  10501   
2018 住院       18     164    141     19    15     19    39     32     87   
     門診     1902   12178   7994   7297   740   1554  1274   2367   1802   

縣市           屏東縣  ...      新竹市    新竹縣    桃園市   澎湖縣   花蓮縣    苗栗縣  連江縣   金門縣  \
年    就診類別         ...                                                        
2008 住院       36  ...      158     31    152     0    60      2    1    47   
     門診     8292  ...     7864  10312  34864  1526  2605  10261   81  1694   
2009 住院      136  ...      243     29    728     0   130     18    0    17   
     門診     7758  ...     5540   7781  24582  2211  3159   7701   90  1458   
2010 住院      228  ...      545    220   1838     1   122    202    1    25   
     門診    19609  ...    23046  26741  84853  2581  6415  19104  142  3098   
2011 住院      138  ...      277     96   1188     2   106     24    0    30   
     門診     9455  ...    10525  11189  45576  1117  3552   8617   20  1061   
2012 住院      199  ...      327     93   1152     1   101    101    0    25   
     門診    11674  ...    13669  14022  48974  1036  4122  12364   27   905   
2013 住院      251  ...      323    139   1310     0   146     51    3    29   
     門診    15220  ...    14375  14156  52461  2435  6079  12122  257  3701   
2014 住院      104  ...      268    126    919     0    67     43    1    18   
     門診    12167  ...    14508  13611  52090  2318  3418   9667   76  2930   
2015 住院       97  ...      374    150    906     0    72    163    1    13   
     門診    14616  ...    17465  14879  51818   946  5720  10564   69  1663   
2016 住院      159  ...      237     78    875     0    55    240    1    10   
     門診    13282  ...    17471  13961  56366  2717  5488  12979  152  2507   
2017 住院      226  ...      236     81    986     0    51    146    0     7   
     門診     9741  ...    13414  10801  49634   844  4384   9053   66  1122   
2018 住院       47  ...       49     17    151     0    19     38    2     4   
     門診     1806  ...     3256   2930   9044   162   880   1596   36   527   

縣市           雲林縣    高雄市  
年    就診類別                
2008 住院       10    291  
     門診    14893  34811  
2009 住院        8    620  
     門診    11097  23736  
2010 住院       54   1557  
     門診    30818  78846  
2011 住院       61    698  
     門診    19495  34705  
2012 住院       20    783  
     門診    15852  39225  
2013 住院       

## Let's check the above one-line code for YiLan : 

In [17]:
d=dict()
for i in range(leng):
    key=df['年'][i]
    d[key]=0
    
for i in range(leng):
    if df['縣市'][i]=='宜蘭縣':
        key=df['年'][i]
        d[key]=d[key]+df['腸病毒健保就診人次'][i]
print(d)

{'2008': 7301, '2009': 7339, '2010': 15705, '2011': 7550, '2012': 10368, '2013': 11486, '2014': 11816, '2015': 11529, '2016': 11781, '2017': 10905, '2018': 1889}


In [20]:
dfm=df.pivot_table(values='腸病毒健保就診人次',index=['年'],columns='縣市',aggfunc='sum')
dfm['宜蘭縣']

年
2008     7301
2009     7339
2010    15705
2011     7550
2012    10368
2013    11486
2014    11816
2015    11529
2016    11781
2017    10905
2018     1889
Name: 宜蘭縣, dtype: int64